In part 1, I attempt to get data from the main chart that details Super Bowl championships from the following Wikipedia page:
https://en.wikipedia.org/wiki/List_of_Super_Bowl_champions. While I could use beautifulsoup to scrape this data directly, Wikipedia has an API with community support which is better to use. In short, there is less fear that Wikipedia might impose a ban because of webscraping. 


In [ ]:
!pip install wikipedia
import wikipedia
from bs4 import BeautifulSoup
import pandas as pd

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=d90bd73632eb71a784e5fbab48c53620ee1cda89a13faa26d374f2b6ad65508c
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


Using the Wikipedia module we can get the html content of a page which we may then convert to a BeautifulSoup object to filter through data we actually need.

In [ ]:
wikipedia.set_rate_limiting(True)

In [ ]:
parsed_html = BeautifulSoup(wikipedia.WikipediaPage("List of Super Bowl champions").html())

In [ ]:
superbowl_table = parsed_html.body.find('table', attrs={'class':'sortable'})

In [ ]:

data_dictionary = {}
# get the column headers of the Wikipedia table
for name in superbowl_table.findAll("th")[0:9]:
    data_dictionary[name.text.rstrip()] = []
# as shown below, all headers were filled approriately, just need to fill with
# data now
data_dictionary

{'Attendance': [],
 'City': [],
 'Date/Season': [],
 'Game': [],
 'Losing team': [],
 'Referee': [],
 'Score': [],
 'Venue': [],
 'Winning team': []}

In [ ]:
# from the superbowl object we can get all the remaining data and pass it 
# directly into our earlier defined dictionary.
for row in superbowl_table.findAll("tr"):
  cells = row.findAll("td")
  if len(cells) == 10:
    for key in enumerate(data_dictionary):
      data_dictionary[key[1]].append(cells[key[0]].find(text=True).rstrip())

In [ ]:
superbowl_history_df = pd.DataFrame.from_dict(data_dictionary)

In [ ]:
# after creating the dataframe we have the following:
superbowl_history_df.head(5).append(superbowl_history_df.tail(5))

,Game,Date/Season,Winning team,Score,Losing team,Venue,City,Attendance,Referee
0,I,"January 15, 1967",Green Bay Packers,35–10,Kansas City Chiefs,Los Angeles Memorial Coliseum,"Los Angeles, California","61,946",Norm Schachter
1,II,"January 14, 1968",Green Bay Packers,33–14,Oakland Raiders,Miami Orange Bowl,"Miami, Florida","75,546",Jack Vest
2,III,"January 12, 1969",New York Jets,16–7,Baltimore Colts,Miami Orange Bowl,"Miami, Florida","75,389",Tom Bell
3,IV,"January 11, 1970",Kansas City Chiefs,23–7,Minnesota Vikings,Tulane Stadium,"New Orleans, Louisiana","80,562",John McDonough
4,V,"January 17, 1971",Baltimore Colts,16–13,Dallas Cowboys,Miami Orange Bowl,"Miami, Florida","79,204",Norm Schachter
54,LV,"February 7, 2021",Tampa Bay Buccaneers,31–9,Kansas City Chiefs,Raymond James Stadium,"Tampa, Florida","24,835",Carl Cheffers
55,LVI,"February 13, 2022",Los Angeles Rams,23–20,Cincinnati Bengals,SoFi Stadium,"Inglewood, California","70,048",Ron Torbert
56,LVII,"February 12, 2023",X 2023,—,To be determined,State Farm Stadium,"Glendale, Arizona",TBD,
57,LVIII,"February 11, 2024",X 2024,—,To be determined,Allegiant Stadium,"Paradise, Nevada",TBD,
58,LIX,"February 9, 2025",X 2025,—,To be determined,Caesars Superdome,"New Orleans, Louisiana",TBD,


In [ ]:
superbowl_history_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Game          59 non-null     object
 1   Date/Season   59 non-null     object
 2   Winning team  59 non-null     object
 3   Score         59 non-null     object
 4   Losing team   59 non-null     object
 5   Venue         59 non-null     object
 6   City          59 non-null     object
 7   Attendance    59 non-null     object
 8   Referee       59 non-null     object
dtypes: object(9)
memory usage: 4.3+ KB


Changes to be made for first dataset:
*   Remove last 3 rows as they are Super Bowls that have yet to occur.
*   Change 'Date/Season' column to just 'Date'. Choice was made to not include 'Season' as Super Bowls are always played on the year after a season e.g. Super Bowl from 2022 had teams from the 2021 season. In short, very easy to know/infer so can do away with removing Season.
*   Convert new 'Date' Column to DateTime object.
*   Split [Score] into [Winner Score] and [Loser Score] -> after that convert new columns into integers
*   Split [City] into [City] and [State] columns
*   Convert values of [Attendance] to integers

In [ ]:
superbowl_history_df.head(5)

,Game,Date/Season,Winning team,Score,Losing team,Venue,City,Attendance,Referee
0,I,"January 15, 1967",Green Bay Packers,35–10,Kansas City Chiefs,Los Angeles Memorial Coliseum,"Los Angeles, California","61,946",Norm Schachter
1,II,"January 14, 1968",Green Bay Packers,33–14,Oakland Raiders,Miami Orange Bowl,"Miami, Florida","75,546",Jack Vest
2,III,"January 12, 1969",New York Jets,16–7,Baltimore Colts,Miami Orange Bowl,"Miami, Florida","75,389",Tom Bell
3,IV,"January 11, 1970",Kansas City Chiefs,23–7,Minnesota Vikings,Tulane Stadium,"New Orleans, Louisiana","80,562",John McDonough
4,V,"January 17, 1971",Baltimore Colts,16–13,Dallas Cowboys,Miami Orange Bowl,"Miami, Florida","79,204",Norm Schachter


In [ ]:
# remove last 3 rows by using indices
superbowl_history_df.drop([56, 57, 58], inplace=True)

Changes to be made for first dataset:
*   ~~Remove last 3 rows as they are Super Bowls that have yet to occur.~~
*   Change 'Date/Season' column to just 'Date'. Choice was made to not include 'Season' as Super Bowls are always played on the year after a season e.g. Super Bowl from 2022 had teams from the 2021 season. In short, very easy to know/infer so can do away with removing Season.
*   Convert new 'Date' Column to DateTime object.
*   Split [Score] into [Winner Score] and [Loser Score] -> after that convert new columns into integers
*   Split [City] into [City] and [State] columns
*   Convert values of [Attendance] to integers

In [ ]:
# rename column and change to date-time object
superbowl_history_df.rename(columns={"Date/Season": "Date"}, inplace=True)
superbowl_history_df["Date"] = pd.to_datetime(superbowl_history_df['Date'])
superbowl_history_df.head()

,Game,Date,Winning team,Score,Losing team,Venue,City,Attendance,Referee
0,I,1967-01-15,Green Bay Packers,35–10,Kansas City Chiefs,Los Angeles Memorial Coliseum,"Los Angeles, California","61,946",Norm Schachter
1,II,1968-01-14,Green Bay Packers,33–14,Oakland Raiders,Miami Orange Bowl,"Miami, Florida","75,546",Jack Vest
2,III,1969-01-12,New York Jets,16–7,Baltimore Colts,Miami Orange Bowl,"Miami, Florida","75,389",Tom Bell
3,IV,1970-01-11,Kansas City Chiefs,23–7,Minnesota Vikings,Tulane Stadium,"New Orleans, Louisiana","80,562",John McDonough
4,V,1971-01-17,Baltimore Colts,16–13,Dallas Cowboys,Miami Orange Bowl,"Miami, Florida","79,204",Norm Schachter


Changes to be made for first dataset:
*   ~~Remove last 3 rows as they are Super Bowls that have yet to occur.~~
*   ~~Change 'Date/Season' column to just 'Date'. Choice was made to not include 'Season' as Super Bowls are always played on the year after a season e.g. Super Bowl from 2022 had teams from the 2021 season. In short, very easy to know/infer so can do away with removing Season.~~
*   ~~Convert new 'Date' Column to DateTime object.~~
*   Split [Score] into [Winner Score] and [Loser Score] -> after that convert new columns into integers
*   Split [City] into [City] and [State] columns
*   Convert values of [Attendance] to integers

In [ ]:
# Score is in XX-YY format -> strip and seperate
winner_score = []
loser_score = []
for score in superbowl_history_df["Score"]:
  scores = score.split("–")
  winner_score.append(scores[0])
  loser_score.append(scores[1])
# re-write column, rename it, and create new column
superbowl_history_df["Score"] = winner_score
superbowl_history_df.rename(columns={"Score": "Winner Score"}, inplace=True)
superbowl_history_df.insert(loc=5, column="Loser Score", value=loser_score)

In [ ]:
superbowl_history_df.head().append(superbowl_history_df.tail(5))

,Game,Date,Winning team,Winner Score,Losing team,Loser Score,Venue,City,Attendance,Referee
0,I,1967-01-15,Green Bay Packers,35,Kansas City Chiefs,10,Los Angeles Memorial Coliseum,"Los Angeles, California","61,946",Norm Schachter
1,II,1968-01-14,Green Bay Packers,33,Oakland Raiders,14,Miami Orange Bowl,"Miami, Florida","75,546",Jack Vest
2,III,1969-01-12,New York Jets,16,Baltimore Colts,7,Miami Orange Bowl,"Miami, Florida","75,389",Tom Bell
3,IV,1970-01-11,Kansas City Chiefs,23,Minnesota Vikings,7,Tulane Stadium,"New Orleans, Louisiana","80,562",John McDonough
4,V,1971-01-17,Baltimore Colts,16,Dallas Cowboys,13,Miami Orange Bowl,"Miami, Florida","79,204",Norm Schachter
51,LII,2018-02-04,Philadelphia Eagles,41,New England Patriots,33,U.S. Bank Stadium,"Minneapolis, Minnesota","67,612",Gene Steratore
52,LIII,2019-02-03,New England Patriots,13,Los Angeles Rams,3,Mercedes-Benz Stadium,"Atlanta, Georgia","70,081",John Parry
53,LIV,2020-02-02,Kansas City Chiefs,31,San Francisco 49ers,20,Hard Rock Stadium,"Miami Gardens, Florida","62,417",Bill Vinovich
54,LV,2021-02-07,Tampa Bay Buccaneers,31,Kansas City Chiefs,9,Raymond James Stadium,"Tampa, Florida","24,835",Carl Cheffers
55,LVI,2022-02-13,Los Angeles Rams,23,Cincinnati Bengals,20,SoFi Stadium,"Inglewood, California","70,048",Ron Torbert


In [ ]:
# there are 56 rows so we want to check that all values are indeed numbers ~ we
# could do this by just looking at each row but better to do so programatically
display(superbowl_history_df["Winner Score"].str.isnumeric().value_counts())
display(superbowl_history_df["Loser Score"].str.isnumeric().value_counts())
# from this we see that the [Winner Score] column is fine but not for [Loser Score]
# i.e. we have one non integer value ~ use regex to remove said value (was just the
# "(" character)
print("After removing non-numeric")
# all of them are now numeric meaning we can convert the columns to number type
superbowl_history_df["Loser Score"] = superbowl_history_df["Loser Score"].str.extract('(\d+)', expand=False)
superbowl_history_df["Loser Score"].str.isnumeric().value_counts()

True    56
Name: Winner Score, dtype: int64

True    56
Name: Loser Score, dtype: int64

True    56
Name: Loser Score, dtype: int64

In [ ]:
superbowl_history_df["Winner Score"] = pd.to_numeric(superbowl_history_df["Winner Score"])
superbowl_history_df["Loser Score"] = pd.to_numeric(superbowl_history_df["Loser Score"])

In [ ]:
superbowl_history_df.head().append(superbowl_history_df.tail(5))

,Game,Date,Winning team,Winner Score,Losing team,Loser Score,Venue,City,Attendance,Referee
0,I,1967-01-15,Green Bay Packers,35,Kansas City Chiefs,10,Los Angeles Memorial Coliseum,"Los Angeles, California","61,946",Norm Schachter
1,II,1968-01-14,Green Bay Packers,33,Oakland Raiders,14,Miami Orange Bowl,"Miami, Florida","75,546",Jack Vest
2,III,1969-01-12,New York Jets,16,Baltimore Colts,7,Miami Orange Bowl,"Miami, Florida","75,389",Tom Bell
3,IV,1970-01-11,Kansas City Chiefs,23,Minnesota Vikings,7,Tulane Stadium,"New Orleans, Louisiana","80,562",John McDonough
4,V,1971-01-17,Baltimore Colts,16,Dallas Cowboys,13,Miami Orange Bowl,"Miami, Florida","79,204",Norm Schachter
51,LII,2018-02-04,Philadelphia Eagles,41,New England Patriots,33,U.S. Bank Stadium,"Minneapolis, Minnesota","67,612",Gene Steratore
52,LIII,2019-02-03,New England Patriots,13,Los Angeles Rams,3,Mercedes-Benz Stadium,"Atlanta, Georgia","70,081",John Parry
53,LIV,2020-02-02,Kansas City Chiefs,31,San Francisco 49ers,20,Hard Rock Stadium,"Miami Gardens, Florida","62,417",Bill Vinovich
54,LV,2021-02-07,Tampa Bay Buccaneers,31,Kansas City Chiefs,9,Raymond James Stadium,"Tampa, Florida","24,835",Carl Cheffers
55,LVI,2022-02-13,Los Angeles Rams,23,Cincinnati Bengals,20,SoFi Stadium,"Inglewood, California","70,048",Ron Torbert


In [ ]:
superbowl_history_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Game          56 non-null     object        
 1   Date          56 non-null     datetime64[ns]
 2   Winning team  56 non-null     object        
 3   Winner Score  56 non-null     int64         
 4   Losing team   56 non-null     object        
 5   Loser Score   56 non-null     int64         
 6   Venue         56 non-null     object        
 7   City          56 non-null     object        
 8   Attendance    56 non-null     object        
 9   Referee       56 non-null     object        
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 4.8+ KB


Changes to be made for first dataset:
*   ~~Remove last 3 rows as they are Super Bowls that have yet to occur.~~
*   ~~Change 'Date/Season' column to just 'Date'. Choice was made to not include 'Season' as Super Bowls are always played on the year after a season e.g. Super Bowl from 2022 had teams from the 2021 season. In short, very easy to know/infer so can do away with removing Season.~~
*   ~~Convert new 'Date' Column to DateTime object.~~
*   ~~Split [Score] into [Winner Score] and [Loser Score] -> after that convert new columns into integers~~
*   Split [City] into [City] and [State] columns
*   Convert values of [Attendance] to integers

In [ ]:
# city is in "CITY, STATE" format -> strip and seperate
cities = []
states = []
for location in superbowl_history_df["City"]:
  locations = location.split(", ")
  
  cities.append(locations[0])
  states.append(locations[1])

superbowl_history_df["City"] = cities
superbowl_history_df.insert(loc=8, column="State", value=states)

In [ ]:
superbowl_history_df.head().append(superbowl_history_df.tail(5))

,Game,Date,Winning team,Winner Score,Losing team,Loser Score,Venue,City,State,Attendance,Referee
0,I,1967-01-15,Green Bay Packers,35,Kansas City Chiefs,10,Los Angeles Memorial Coliseum,Los Angeles,California,"61,946",Norm Schachter
1,II,1968-01-14,Green Bay Packers,33,Oakland Raiders,14,Miami Orange Bowl,Miami,Florida,"75,546",Jack Vest
2,III,1969-01-12,New York Jets,16,Baltimore Colts,7,Miami Orange Bowl,Miami,Florida,"75,389",Tom Bell
3,IV,1970-01-11,Kansas City Chiefs,23,Minnesota Vikings,7,Tulane Stadium,New Orleans,Louisiana,"80,562",John McDonough
4,V,1971-01-17,Baltimore Colts,16,Dallas Cowboys,13,Miami Orange Bowl,Miami,Florida,"79,204",Norm Schachter
51,LII,2018-02-04,Philadelphia Eagles,41,New England Patriots,33,U.S. Bank Stadium,Minneapolis,Minnesota,"67,612",Gene Steratore
52,LIII,2019-02-03,New England Patriots,13,Los Angeles Rams,3,Mercedes-Benz Stadium,Atlanta,Georgia,"70,081",John Parry
53,LIV,2020-02-02,Kansas City Chiefs,31,San Francisco 49ers,20,Hard Rock Stadium,Miami Gardens,Florida,"62,417",Bill Vinovich
54,LV,2021-02-07,Tampa Bay Buccaneers,31,Kansas City Chiefs,9,Raymond James Stadium,Tampa,Florida,"24,835",Carl Cheffers
55,LVI,2022-02-13,Los Angeles Rams,23,Cincinnati Bengals,20,SoFi Stadium,Inglewood,California,"70,048",Ron Torbert


Changes to be made for first dataset:
*   ~~Remove last 3 rows as they are Super Bowls that have yet to occur.~~
*   ~~Change 'Date/Season' column to just 'Date'. Choice was made to not include 'Season' as Super Bowls are always played on the year after a season e.g. Super Bowl from 2022 had teams from the 2021 season. In short, very easy to know/infer so can do away with removing Season.~~
*   ~~Convert new 'Date' Column to DateTime object.~~
*   ~~Split [Score] into [Winner Score] and [Loser Score] -> after that convert new columns into integers~~
*   ~~Split [City] into [City] and [State] columns~~
*   Convert values of [Attendance] to integers

In [ ]:
# Before converting Attendance values into numbers, just need to get rid of commas
superbowl_history_df['Attendance']=superbowl_history_df['Attendance'].str.replace(',','')

In [ ]:
# check we can convert to numbers; all came up true so yes we can
display(superbowl_history_df["Attendance"].str.isnumeric().value_counts())

True    56
Name: Attendance, dtype: int64

In [ ]:
superbowl_history_df["Attendance"] = pd.to_numeric(superbowl_history_df["Attendance"])

In [ ]:
superbowl_history_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 55
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Game          56 non-null     object        
 1   Date          56 non-null     datetime64[ns]
 2   Winning team  56 non-null     object        
 3   Winner Score  56 non-null     int64         
 4   Losing team   56 non-null     object        
 5   Loser Score   56 non-null     int64         
 6   Venue         56 non-null     object        
 7   City          56 non-null     object        
 8   State         56 non-null     object        
 9   Attendance    56 non-null     int64         
 10  Referee       56 non-null     object        
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 5.2+ KB


In [ ]:
superbowl_history_df.head()

,Game,Date,Winning team,Winner Score,Losing team,Loser Score,Venue,City,State,Attendance,Referee
0,I,1967-01-15,Green Bay Packers,35,Kansas City Chiefs,10,Los Angeles Memorial Coliseum,Los Angeles,California,61946,Norm Schachter
1,II,1968-01-14,Green Bay Packers,33,Oakland Raiders,14,Miami Orange Bowl,Miami,Florida,75546,Jack Vest
2,III,1969-01-12,New York Jets,16,Baltimore Colts,7,Miami Orange Bowl,Miami,Florida,75389,Tom Bell
3,IV,1970-01-11,Kansas City Chiefs,23,Minnesota Vikings,7,Tulane Stadium,New Orleans,Louisiana,80562,John McDonough
4,V,1971-01-17,Baltimore Colts,16,Dallas Cowboys,13,Miami Orange Bowl,Miami,Florida,79204,Norm Schachter


In [ ]:
# can download the file uncode as necessary
#from google.colab import files
#superbowl_history_df.to_excel("superbowl.xlsx")
#files.download("superbowl.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
superbowl_history_df

Done and data looks nice to work with now. For part 2 will attempt to get more specific in game data such

*   as points per quarter (and if applicable Overtime)
*   the MVP: player and position
*   Head Coach
*   Favorite Team Going into match
*   Cost of 30 second Commercial: Cost at time and adjusted for inflation
